Будем использовать физическими величинами с измерениями. Модуль PhysicalQuantities сломался, не обновлялся два года, поэтому придется попробовать новое.

In [8]:
from pint import UnitRegistry
from uncertainties import ufloat

In [9]:
ureg = UnitRegistry()
length = ufloat(10, 0.5) * ureg.meter
print(length)  # (10 ± 0.5) m


10.0+/-0.5 meter


In [12]:
H_отсчета = ufloat(200, 10) * ureg.kilometer
H_отсчета

<Quantity(200.000+/-10.000, 'kilometer')>

In [10]:
P_отсчета =  ufloat(8.53, 1)* 1e-5 * ureg.pascal
P_отсчета

<Quantity((8.53000+/-1.00000)e-05, 'pascal')>

In [13]:
H = ufloat(60, 10) * ureg.kilometer
H

<Quantity(60.0000+/-10.0000, 'kilometer')>

In [15]:
σ = ufloat(8, 3) * ureg.angstrom * ureg.angstrom
σ

<Quantity(8.00000+/-3.00000, 'angstrom ** 2')>

In [16]:
T_отсчета = ufloat(854.4, 200) * ureg.kelvin
T_отсчета

<Quantity(854.400+/-200.000, 'kelvin')>

In [17]:
T_экзосферы = ufloat(1000, 200) * ureg.kelvin
T_экзосферы

<Quantity(1000.00+/-200.00, 'kelvin')>

In [30]:
Tstart=ureg.k * ureg.kelvin


In [31]:
h_экз = ureg.k * T_экзосферы /  σ / H
h_экз.to_base_units()

ValueError: Cannot operate with Unit and Quantity of different registries.

In [25]:
from pint import UnitRegistry
ureg = UnitRegistry()
k = ureg.boltzmann_constant
print(k.magnitude)  


AttributeError: 'Unit' object has no attribute 'magnitude'